# RNN (work in progress)


> **why we cannot used ANN or CNN for sequential data ?**

- the text data is not of same size.(each example can have the different lenght).

- and cnn or ann canot deal with the input of diff-diff lenght.

- what if convert the every smaple into same size but the amount of unsessary data that we gonna create is crazy not computer efficient at all.

- and also at the time of inferece if we input have diff length then the zero padding is also not gonna work.(i.e larger than out highest our training sample lenght)

- and one thing in ANN or CNN we pass the data at a same time this can cuase the big probelm. 

- cuase the meaining of sentence cann be change or look liek randome noise caues in text the most imp thing is the squence of the words.

- by design the ann cannot remember anything due it we lose the semantic meaning.

> **How RNN solve this problem ?** 

- rnn can handle the sequential data.(mostly text data).

- rnn has this feature due to it can remember the previous features.

> **RNN-Architecure**

- 